# INSTALLATION DE DEPENDANCES

In [27]:
!pip install requests
!pip install apscheduler
!pip install pandas

You should consider upgrading via the 'c:\users\hp omen\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\hp omen\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\hp omen\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# IMPORTS NECESSAIRES POUR TRAVAILLER


In [46]:
import requests
import json
import pandas as pd
from apscheduler.schedulers.blocking import BlockingScheduler
import os

In [7]:
API_KEY = "4842ad786edd4e3a2dfcc8d1392569911a5cdeceeb76fa150a2048ee83f577a5"

In [32]:
species_df = pd.read_excel("scientificNames.xlsx")
species_names = list(species_df['Espèce'])

# REQUEST DATA FUNCTION

In [17]:
def request_datas(url,parameters={}):
    response = requests.get(url, params=parameters)
    try:
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 403:
            print("Non autorisé")
            return None
        else:
            return None
    except ConnectionError:
        print("connection error retry later")

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# API CALLS TO GET DATAS

In [18]:
print(request_datas('https://jsonplaceholder.typicode.com/todos/1'))

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}


## GOOGLE SCHOLAR

In [19]:
# Test request
scholar_google= None
scholar_google = request_datas("https://serpapi.com/search.json",{"engine":"google_scholar", "q":"plants,Cameroon,articles, Bulbophyllum Thouars","api_key":API_KEY})
jprint(scholar_google)
# If you see null as result that means that API_Key has expired

null


### TRAITEMENT DES DATAS POUR SCHOLAR

In [20]:
'''
la structure des données qui nous intresse: 
"organic_results" : [
  {
            "link": 
            "position": 0,
            "publication_info": {
                "summary": 
            },
            "snippet": 
            "title": 
        },
]
"related_searches" : [{"link","query"}]

"serpapi_pagination" : {
  "current":
  "next":
  "other_pages"
}
'''

'\nla structure des données qui nous intresse: \n"organic_results" : [\n  {\n            "link": \n            "position": 0,\n            "publication_info": {\n                "summary": \n            },\n            "snippet": \n            "title": \n        },\n]\n"related_searches" : [{"link","query"}]\n\n"serpapi_pagination" : {\n  "current":\n  "next":\n  "other_pages"\n}\n'

In [24]:
def search_scholar(plant : str):
    """This function returns a list of json corresponding to request search results
        args : plant_name[str]
    """
    results = None
    results_data = []
    results = request_datas("https://serpapi.com/search.json",{"engine":"google_scholar", "q": plant ,"api_key":API_KEY})
    if (results is not None ) :
        if len(results.keys())>0 : 
            if ("error" not in results.keys()) :
                while (results is not None) :
                    if "organic_results" in results.keys() :
                        results_data.append(results["organic_results"])
                    if ("serpapi_pagination" in results.keys()) :
                        if ('next' in results["serpapi_pagination"].keys()):
                            results = request_datas(results["serpapi_pagination"]["next"],{"api_key":API_KEY})
                        else :
                            break
                    else :
                        break
    return results_data

In [35]:
def searchLine(listOfNames) :
    """This function return in a dataFrame all the request results for a part of plant names present in listofNames"""
    n = len(listOfNames)
    df =  None
    plantnames = []
    articledescr = []
    articletitle = []  
    articlelink = []
    for k in range(n) :
        name = listOfNames[k]
        search = search_scholar(name)
        if len(search)>0 :
            for s in search:
                for elem in s :
                    if (elem is not None) & ('snippet' in elem.keys()) :
                        plantnames.append(name)
                        articledescr.append(elem["snippet"])
                        articletitle.append(elem["title"])
                        articlelink.append(elem["link"])
        else :
            break
    df = pd.DataFrame({"plant name" : plantnames, "article title" : articletitle, "article description":articledescr, "link" : articlelink})
    return df 


In [50]:
counter = 827
file_counter = 1
def continuous_search() :
    global counter
    if counter < len(species_names) :
        species_search_list = species_names[counter : ]
        gbif_df = searchLine(species_search_list)
        if gbif_df is not None :
            global file_counter
            gbif_df.to_excel("gbif"+str(file_counter)+".xlsx") 
            file_counter += 1
            counter += len(gbif_df)
            print(counter)

In [ ]:
if __name__ == '__main__':
    scheduler = BlockingScheduler()
    scheduler.add_job(continuous_search, 'interval', hours = 1.15)
    print('Press Ctrl+{0} to exit'.format('Break' if os.name == 'nt' else 'C'))
    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        pass